In [1]:
# Import statements
import pandas as pd
import fasttext as ft
import faiss
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from datetime import datetime, timedelta
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/CS/pvp0001/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/CS/pvp0001/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [116]:
# Reading .csv as pandas dataframe
dir_name = "/raid/Praveen_Stuff/Eluvio_DS_Challenge.csv"
df = pd.read_csv(dir_name)
print(df.head)

<bound method NDFrame.head of         time_created date_created  up_votes  down_votes  \
0         1201232046   2008-01-25         3           0   
1         1201232075   2008-01-25         2           0   
2         1201232523   2008-01-25         3           0   
3         1201233290   2008-01-25         1           0   
4         1201274720   2008-01-25         4           0   
...              ...          ...       ...         ...   
509231    1479816764   2016-11-22         5           0   
509232    1479816772   2016-11-22         1           0   
509233    1479817056   2016-11-22         1           0   
509234    1479817157   2016-11-22         1           0   
509235    1479817346   2016-11-22         1           0   

                                                                                           title  \
0                                                              Scores killed in Pakistan clashes   
1                                                            

In [117]:
# Generating unique id's for our data
ids = list()
length = df.shape[0]
x = 1
for i in range(length):
    ids.append(x)
    x+=1

In [118]:
# Writing unique ids to our dataframe
df['unique_id'] = ids
# Get the values of every column as a list
ids = df['unique_id'].values.tolist()
time_created = df['time_created'].values.tolist()
date_created = df['date_created'].values.tolist()
author = df['author'].values.tolist()
titles =  df['title'].values.tolist()

In [121]:
def remove_stopwords(titles):
    '''
    Function to remove stop words given a list of senetences.
    
    senetences : List of strings
    
    returns : List of List of strings
    '''
    # Tokenize the sentence
    word_tokens = []
    for each_title in titles:
        word_token = word_tokenize(each_title) 
        word_tokens.append(word_token)
    # Remove stop words
    stop_words = set(stopwords.words('english'))
    filtered_sentences = []
    for word_token in word_tokens:
        filtered_sentence = [] 
        for w in word_token:
            if w not in stop_words: 
                filtered_sentence.append(w) 
        filtered_sentences.append(filtered_sentence)
    return filtered_sentences

In [122]:
filtered_sentences = remove_stopwords(titles)

In [123]:
def stemSentence(sentences):
    '''
    Function to stem sentences given a list of senetences.
    
    senetences : List of List of strings
    
    returns : List of sentences
    '''
    porter = PorterStemmer()
    stem_sentences = []
    for sentence in sentences:
        stem_sentence=[]
        for word in sentence:
            stem_sentence.append(porter.stem(word))
            stem_sentence.append(" ")
        stem_sentences.append("".join(stem_sentence))
    return stem_sentences

In [124]:
stem_sentences = stemSentence(filtered_sentences)

In [166]:
print(stem_sentences[100])

japan say russian bomber violat it airspac - new york time 


In [125]:
# Function to write to text
# We are appending Time, Date, Author to title files
def write_text(ids_path, titles_path, ids, titles, time_created, date_created, author):
    '''
    Function to write to a file
    
    ids_path : path with filename.txt to store unique id's
    titles_path : path with filename.txt to store titles appended with time, date and author
    id's: List of Unique id's, whose indices correspond to the indices of title
    titles: List of Titles
    time_created: List of time_created, whose indices correspond to the indices of title
    date_created : List of date_created, whose indices correspond to the indices of title
    author = List of author, whose indices correspond to the indices of title
    
    returns: None
    '''
    f1 = open(ids_path, 'w')
    f2 = open(titles_path, 'w')
    for i in range(length):
        f1.write(str(ids[i]))
        f1.write("\n")
        f2.write("time: ")
        f2.write(str(time_created[i]))
        f2.write(" ")
        f2.write("date: ")
        f2.write(str(date_created[i]))
        f2.write(" ")
        f2.write("author: ")
        f2.write(author[i])
        f2.write(" ")
        f2.write("title: ")
        f2.write(titles[i])
        f2.write("\n")
    f1.close()
    f2.close()

In [126]:
ids_path = "idx_processed_appended.txt"
titles_path = "title_processed_appended.txt"
# Writes Id's and titles to the text file
write_text(ids_path, titles_path, ids, stem_sentences, time_created, date_created, author)

In [81]:
# Training an Fast Text model to create embeddings
embedding_dim = 100
window_size = 5
min_word_count = 5
EPOCH = 25
model_name = "ft_model_em100_ws5_mwc5_ep25_preprocessed_appended.bin"
# Uncomment,if you want to train the fast text model
# ft_model = ft.train_unsupervised(input=titles_path, dim=embedding_dim, epoch=EPOCH, model="skipgram")
# Uncomment,if you want to save the fast text model
# ft_model.save_model(model_name)

In [82]:
ft_model = ft.load_model(model_name)

In [83]:
# We are opening the unique id's and preprocessed_titles with time, date and author file
desc_file = open(titles_path, 'r')
id_file = open(ids_path, 'r')
desc_Lines = desc_file.readlines()
id_lines = id_file.readlines()
print(len(id_lines))

509236


In [84]:
# We are creating the embedding matrix using the trained Fast text model
job_id_list = list()
embedding_list = list()
i = 0
for job_id,job_desc in zip(id_lines,desc_Lines):
    embedding = ft_model.get_sentence_vector(job_desc.rstrip())
    job_id_list.append(int(job_id))
    embedding_list.append(embedding)
    i+=1

In [159]:
len(job_id_list)

509236

In [85]:
def recommend(job_ids, embeddings, target_ids, num_recs, vec_dim):
    '''
    Uses facebook FAISS library to perform L2 similarity search with 100-d vector embeddings
    
    job_ids: list of jobs_ids, whose indices correspond to embedding indices
    embeddings: numpy array of 100-d vectors, whose indices correspond to the indices of job_ids
    target_ids: list of target ids
    num_recs: the number of recommendations to make
    
    returns: job dictionary with key = job id and value = embedding
    '''
    #Get the corresponding embeddings for each target ID
    #and convert to numpy array for FAISS
    queries = []
    for tid in target_ids:
        queries.append(embeddings[job_ids.index(tid)])
    search_queries = np.asarray(queries)
    #Brute force, L2 distance search index.
    index = faiss.IndexFlatL2(vec_dim) #build the index
    index.add(embeddings)
    D, I = index.search(search_queries, num_recs)
    print("Finding the nearest neighbor of the following query: ")
    similar_jobs = {}
    #index is a list containing indexes corresponding to number of reccomendations requested
    for index in I:
        for element in index:
            similar_jobs.update({job_ids[element]: embeddings[element]})
            
    return similar_jobs

In [86]:
embedding_list = np.asarray(embedding_list)
target_job_ids_list = [1]
similar_job_embeddings = recommend(job_id_list, embedding_list, target_job_ids_list, 20, 100)

[array([ 0.04222713, -0.03767658, -0.11204252, -0.03170907,  0.02707273,
       -0.02723083, -0.0564267 , -0.00905808,  0.00707193,  0.0405337 ,
       -0.04255448, -0.08341026,  0.0723864 , -0.07140297, -0.06992448,
        0.03960486, -0.04777391,  0.01554316, -0.01983578, -0.02647545,
        0.0443822 , -0.00086667,  0.06553064,  0.06683465,  0.03487814,
       -0.01339015, -0.01525639,  0.09661102,  0.00549085,  0.05244684,
       -0.10818604, -0.12197697,  0.03678501, -0.05858905,  0.00154539,
        0.05988876, -0.01616365, -0.04995066,  0.0232853 , -0.06058578,
       -0.04624536,  0.02377324,  0.01242548,  0.10018274,  0.08303244,
        0.0265761 , -0.00770316, -0.02071305, -0.13202055, -0.05777887,
       -0.09516828, -0.00798994,  0.04117286,  0.01577277,  0.00315286,
        0.00434462, -0.00741288, -0.02544191, -0.07644507, -0.04221267,
        0.03028675,  0.05331694, -0.00263942,  0.10309516,  0.11012197,
        0.16074981,  0.01297159, -0.02167593,  0.0165487 , -0.0

In [87]:
def get_rec_desc(similar_dict, column_name, df):
    '''
    Finds the recommendation based on similar_job_emebeddings 
    
    similar_dict : Similar job embeddings
    column_name : the name of the column in the dataframe where unique id matches
    df: the dataframe of our work
    returns : target dataframe and recommendation dataframe
    '''
    final_df = pd.DataFrame()
    for target in similar_dict.keys():
        target_df = df.loc[df[column_name] == target]
        if(final_df.empty):
            final_df = df.loc[df[column_name] == target]
        else:
            final_df = pd.concat([final_df, df.loc[df[column_name] == target]])
    # Sort based on the number of up_votes
    final_df = final_df.sort_values(by=['up_votes'], ascending=False)        
    return target_df, final_df

In [88]:
target_df, recs_df = get_rec_desc(similar_job_embeddings, "unique_id", df)

In [89]:
print(recs_df)

        time_created date_created  up_votes  down_votes  \
277480    1400809678   2014-05-23       107           0   
193619    1372620163   2013-06-30        74           0   
215289    1379888846   2013-09-22        40           0   
137482    1348238627   2012-09-21        14           0   
263607    1395417454   2014-03-21        13           0   
160561    1361056495   2013-02-16        13           0   
38953     1246875680   2009-07-06         8           0   
170699    1365008261   2013-04-03         4           0   
118359    1333071968   2012-03-30         3           0   
157457    1359587302   2013-01-30         3           0   
0         1201232046   2008-01-25         3           0   
404205    1444705867   2015-10-13         1           0   
154101    1357940640   2013-01-11         1           0   
333404    1420404408   2015-01-04         1           0   
82277     1301322903   2011-03-28         1           0   
426587    1451351897   2015-12-29         0           0 

In [100]:
# 30 days before Recommendation
from datetime import datetime, timedelta
max_date = df['date_created'].max()
start_date = max_date + timedelta(days=-30)
print(max_date)
print(max_date + timedelta(days=-30))

2016-11-22 00:00:00
2016-10-23 00:00:00


In [141]:
df['date_created'] = pd.to_datetime(df['date_created'], format='%Y-%m-%d')
start_df = df[df['date_created']== start_date]
start_idx = start_df['unique_id'][0:1].item() - 1
print(start_idx)

501610


In [161]:
# This function recommends articles from past 30 days
def recommend_30days(job_ids, new_job_ids, embeddings, new_embeddings, target_ids, num_recs, vec_dim):
    '''
    Uses facebook FAISS library to perform L2 similarity search with 100-d vector embeddings
    
    job_ids: list of jobs_ids, whose indices correspond to embedding indices
    embeddings: numpy array of 100-d vectors, whose indices correspond to the indices of job_ids
    target_ids: list of target ids
    num_recs: the number of recommendations to make
    
    returns: job dictionary with key = job id and value = embedding
    '''
    #Get the corresponding embeddings for each target ID
    #and convert to numpy array for FAISS
    queries = []
    for tid in target_ids:
        queries.append(embeddings[job_ids.index(tid)])
    search_queries = np.asarray(queries)
    #Brute force, L2 distance search index.
    index = faiss.IndexFlatL2(vec_dim) #build the index
    index.add(new_embeddings)
    D, I = index.search(search_queries, num_recs)
    print("Finding the nearest neighbor of the following query: ")
    similar_jobs = {}
    #index is a list containing indexes corresponding to number of reccomendations requested
    for index in I:
        for element in index:
            similar_jobs.update({new_job_ids[element]: embeddings[element]})
            
    return similar_jobs

In [162]:
new_embedding_list = embedding_list[start_idx:]
new_job_id_list = job_id_list[start_idx:]
target_job_ids_list = [1]
similar_job_embeddings_30d = recommend_30days(job_id_list, new_job_id_list, embedding_list, new_embedding_list, target_job_ids_list, 5, 100)

Finding the nearest neighbor of the following query: 


In [163]:
new_target_df, recs_df_30 = get_rec_desc(similar_job_embeddings_30d, "unique_id", df)

In [164]:
print(recs_df_30)

        time_created date_created  up_votes  down_votes  \
502055    1477355704   2016-10-25      1956           0   
502278    1477416451   2016-10-25        16           0   
502546    1477488731   2016-10-26         9           0   
507189    1479185962   2016-11-15         9           0   
504584    1478128730   2016-11-02         8           0   

                                                         title  over_18  \
502055      Forty eight dead in Pakistan police academy attack    False   
502278          Israeli killed by gunfire from Egypt on border    False   
502546  Dozens of civilians abducted and killed in Afghanistan    False   
507189  Kashmir clash: Pakistan says India killed seven troops    False   
504584           13 killed by cross-border shelling in Kashmir    False   

                   author   category  unique_id  
502055            xicomen  worldnews     502056  
502278              IFRIC  worldnews     502279  
502546  middleeastnewsman  worldnews     502